In [1]:
import pandas as pd
import pandas_gbq
import re

In [7]:
df_pthList=pd.read_csv('list_folder.txt',header=None)
a=[]
for i in df_pthList.loc[:,0]:
    if  re.search('Energy_input.+monthly.xls',i):
        if 'P001' in i:
            if 'aseline' in i:
                b=i.replace('\\','/')
                a.append(b)
#print(a)
pth=pd.DataFrame(a)
pth=pth.loc[:,0].str.split(pat='/',expand=True)
pth.head()

,0,1,2,3,4,5,6,7,8,9,10
0,C:,inetpub,wwwroot,trnsysmodel,F00000000000002,1,P001,B1 unit,Haadthip Boiler baseline,Datalogger,Energy_input_monthly.xls
1,C:,inetpub,wwwroot,trnsysmodel,F00000000000002,3,P001,3 units,HAADTHIP Aircomp baseline,Datalogger,Energy_input_monthly.xls
2,C:,inetpub,wwwroot,trnsysmodel,F00000000000003,3,P001,1 unit,KITPAWORN Aircomp baseline,Datalogger,Energy_input_monthly.xls
3,C:,inetpub,wwwroot,trnsysmodel,F00000000000004,3,P001,5 units,KHONBURI Aircom baseline,Datalogger,Energy_input_monthly.xls
4,C:,inetpub,wwwroot,trnsysmodel,F00000000000004,3,P001,5 units,KHONBURI Aircomp baseline,Datalogger,Energy_input_monthly.xls


In [8]:
df_getData=pd.DataFrame()
df_getData['factory']=pth.loc[:,4]
df_getData['system']=pth.loc[:,5]
df_getData['name']=pth.loc[:,8]
df_getData['link']="http://54.254.220.213/trnsysmodel"
for i in range(7):
    df_getData['link']=df_getData['link']+"/"+pth.loc[:,i+4].str.replace(' ','%20')
df_getData.head()

,factory,system,name,link
0,F00000000000002,1,Haadthip Boiler baseline,http://54.254.220.213/trnsysmodel/F00000000000...
1,F00000000000002,3,HAADTHIP Aircomp baseline,http://54.254.220.213/trnsysmodel/F00000000000...
2,F00000000000003,3,KITPAWORN Aircomp baseline,http://54.254.220.213/trnsysmodel/F00000000000...
3,F00000000000004,3,KHONBURI Aircom baseline,http://54.254.220.213/trnsysmodel/F00000000000...
4,F00000000000004,3,KHONBURI Aircomp baseline,http://54.254.220.213/trnsysmodel/F00000000000...


In [9]:
def colToRow(df_temColData,factory,system,name):
    df_tod2=pd.DataFrame(columns=['no','Period','value'])
    month=df_temColData.loc[:,'Period']
    for k in range(df_temColData.shape[1]-1):
        temp=pd.DataFrame(month)
        temp['value']=df_temColData.iloc[:,k+1]
        temp['no']=k+1
        df_tod2=df_tod2.append(temp)
    df_tod2['factory']=factory
    df_tod2['system']=system
    df_tod2['name']=name
    return df_tod2.loc[:,['factory','system','name','no','Period','value']]

df=pd.DataFrame(columns=['factory','system','name','no','Period','value'])
for i in df_getData.index:
    try:
        df_temColData=pd.read_csv(df_getData.loc[i,'link'],sep='\t',skiprows=1,nrows=12)
        df_temColData=df_temColData.dropna(axis=1)
        df_tod=colToRow(df_temColData,df_getData.loc[i,'factory'],df_getData.loc[i,'system'],df_getData.loc[i,'name'])
        df=df.append(df_tod)
        #print(df_getData.loc[i,'system'],df_temColData.shape,df.shape)
    except:
        print(df_getData.loc[i,'factory'],df_getData.loc[i,'system'])

F00000000000010 1
F00000000000011 2
F00000000000011 2
F00000000000014 1
F00000000000017 1
F00000000000021 2
F00000000000021 2
F00000000000021 2
F00000000000021 2
F00000000000021 3
F00000000000025 1
F00000000000027 1
F00000000000029 3
F00000000000034 1


In [11]:
df['system']=df['system'].map({'1':'Boiler','2':'Chiller','3':'Compressor'})
df['system'].value_counts()

Compressor    1644
Chiller        984
Boiler         324
Name: system, dtype: int64

In [12]:
df_factoryID=pd.read_csv("factoryID.txt",header=None)
df_factoryID[0]=df_factoryID[0].str.replace("<option value=\"","")
df_factoryID[0]=df_factoryID[0].str.replace("</option>","")
df_factoryID[0]=df_factoryID[0].str.replace("\">",",")
df_factoryID=df_factoryID[0].str.split(pat=",",expand=True)
df['factory']=df['factory'].map(dict(zip(df_factoryID[0],df_factoryID[1])))
df.head()

,factory,system,name,no,Period,value
0,บริษัท หาดทิพย์ จำกัด (มหาชน),Boiler,Haadthip Boiler baseline,1,January,3.733150e+09
1,บริษัท หาดทิพย์ จำกัด (มหาชน),Boiler,Haadthip Boiler baseline,1,February,3.370747e+09
2,บริษัท หาดทิพย์ จำกัด (มหาชน),Boiler,Haadthip Boiler baseline,1,March,3.728611e+09
3,บริษัท หาดทิพย์ จำกัด (มหาชน),Boiler,Haadthip Boiler baseline,1,April,3.603879e+09
4,บริษัท หาดทิพย์ จำกัด (มหาชน),Boiler,Haadthip Boiler baseline,1,May,3.719391e+09


## Debug ##

In [13]:
df_getData.loc[(df_getData['factory']=="F00000000000023"),:]

,factory,system,name,link
62,F00000000000023,2,Winchance Chiller Aircool baseline,http://54.254.220.213/trnsysmodel/F00000000000...
63,F00000000000023,2,Winchance Chiller Aircool baseline,http://54.254.220.213/trnsysmodel/F00000000000...
64,F00000000000023,3,Winsand Aircom Baseline,http://54.254.220.213/trnsysmodel/F00000000000...


In [30]:
test=df.loc[(df['factory']=="บริษัท วินแชนซ์ ฟู้ดส์ จํากัด")&(df['system']=="Chiller"),:]

In [31]:
test.to_csv("test.csv")

# Upload to GCP #

In [15]:
df.to_gbq(destination_table='FEC.energy_input_monthly',project_id='introcloud01',if_exists='replace')

1it [00:08,  8.84s/it]
